TEST ML_composer 


In [1]:
import os
from GS_composer import *
tf.config.run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    gpu = physical_devices[0]
    print(gpu)
    assert gpu.device_type == 'GPU'

In [3]:
import os
print(os.getcwd())
#os.chdir("F:/project/sugarcane_disease/")
#os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
user_profile = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/data/"
#user_profile = "H:/Data/"
#os.chdir("E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")
#os.chdir("C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")

F:\PhD_Notebook\Code\ML_composer


In [4]:
#tf.config.experimental.get_device_details(gpu)

NameError: name 'gpu' is not defined

In [5]:
from ClassModel import *
from CustomLayers import *

In [6]:
from Functions import *

In [7]:
class p_args:
    
    def __init__(self):
        
        self.ped = user_profile +"sugarcane_disease/subset/disease_subset"
        self.pheno = user_profile +"sugarcane_disease/subset/disease_subset.phen"
        self.mpheno = 1
        self.index = user_profile +"sugarcane_disease/subset/disease_subset.index"
        self.model = "MultiLevel Attention"
        self.annotation = None #user_profile +"sugarcane_disease/subset/disease_subset.anno"
        self.trait = "smut"
        self.output = "../Attention_Test_win"
        
        self.load = None
        self.save = True
        self.plot = True
        self.loss = "mae"
        
        self.round = 1
        self.epoch = 5
        self.rank =False
        self.lr = 0.0001
        self.batch = 32
        self.mean = False
        self.width = 64
        self.depth = 2
        self.quiet = 1
        self.num_heads = 1
        self.residual = True
        self.embedding = 12
        self.locallyConnect = 12
        self.data_type = "numerous"
        self.activation = "linear"
        self.loss = "mae"
        self.classes = 0
        
        
args = p_args()

In [8]:
config = None


In [9]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [10]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [11]:
#composer.get_data(config,args)
composer.args = args

In [12]:
composer.get_data(config,args=args)

2
1    939
3    940
4    940
2    940
5    943
Name: count, dtype: int64
Get genotype shape: (4702, 100)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2
Running data check
GENO
FAM
PHENO
INDEX


In [13]:
#composer.prepare_model()
#composer._model["INIT_MODEL"].init_model()

In [14]:
args.model

'MultiLevel Attention'

In [15]:
index_ref = composer.prepare_cross_validate()

In [16]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [17]:
train_idx,valid_idx = index_ref[0]

In [18]:
composer.prepare_training(train_idx,valid_idx)
#composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1.0
39    3.0
44    3.0
47    2.0
92    3.0
Name: 2, dtype: float64


In [19]:
#composer._raw_data["GENO"].iloc[:,6:].describe()

In [20]:
composer.compose(train_idx,valid_idx)

USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (3047, 100, 1)
USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (896, 100, 1)
Train status:
Epochs:  5
Repeat(Round):  1
Got input shape: (100, 1)
(None, 100, 1)
(None, 10, 12)
From model
(None, 10, 12)
From model
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer_1 (InputLayer)     [(None, 100, 1)]     0           []                               
                                                                                                  
 zero_padding1d (ZeroPadding1D)  (None, 101, 1)      0  

F:\miniconda\envs\tf29\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


96/96 [==============================] - 2s 24ms/step - loss: 16.9719 - acc: 0.0226 - val_loss: 4.2611 - val_acc: 0.0190 - lr: 0.0010
Epoch 2/5
96/96 [==============================] - 2s 23ms/step - loss: 5.4711 - acc: 0.0309 - val_loss: 3.9714 - val_acc: 0.0190 - lr: 0.0010
Epoch 3/5
96/96 [==============================] - 2s 22ms/step - loss: 5.4281 - acc: 0.0309 - val_loss: 4.8062 - val_acc: 0.0190 - lr: 0.0010
Epoch 4/5
96/96 [==============================] - 2s 22ms/step - loss: 5.3527 - acc: 0.0309 - val_loss: 4.1181 - val_acc: 0.0190 - lr: 0.0010
Epoch 5/5
96/96 [==============================] - 2s 21ms/step - loss: 5.3287 - acc: 0.0309 - val_loss: 4.4157 - val_acc: 0.0190 - lr: 0.0010
 - train loss     : 5.328730583190918
 - validation loss: 4.415687561035156
 - loss decrease rate in last 5 epochs: 0.03172125816345215
 - Actual Training epochs:  5
96/96 [==============================] - 1s 5ms/step
Train End.
In-year accuracy (measured as Pearson's correlation) is:  0.1951

INFO:tensorflow:Assets written to: F:\PhD_Notebook\Code\Attention_Test_win/smut_MultiLevelAttention_1\assets


INFO:tensorflow:Assets written to: F:\PhD_Notebook\Code\Attention_Test_win/smut_MultiLevelAttention_1\assets


Model saved.
Resource check:
Total memory: 31.7264 GB
Currently using memory: 0.4682 GB
Ratio of used memory: 41.2000  %
Number of CPU node:  16
Plotting the training process...
F:\PhD_Notebook\Code\Attention_Test_win/smut_MultiLevelAttention_smut
Plotting loss history...
Plot name:  F:\PhD_Notebook\Code\Attention_Test_win/smut_MultiLevelAttention_smut/smut_MultiLevelAttention_1.png
Result:
  Trait      TrainSet ValidSet                Model  Test_Accuracy   
0  smut  [2, 3, 4, 5]      [1]  MultiLevelAttention       0.195138  \

   Valid_Accuracy       mae   Runtime  
0        0.053408  4.415688  0.183333  


In [21]:
composer._model["TRAINED_MODEL"].save("./test_model")

#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

INFO:tensorflow:Assets written to: ./test_model\assets


INFO:tensorflow:Assets written to: ./test_model\assets


Please install GPU version of TF


In [ ]:
print(composer.)

In [ ]:
os.getcwd()

In [ ]:
tf.executing_eagerly()

In [ ]:
print(tf.__version__)

In [ ]:
input_tensor = tf.constant([1, 2, 3, 4, 5])
input_tensor.numpy()

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1